In [2]:
# Fix import path
import sys
import os

# Add the directory containing scraper.py to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..','..')))


In [11]:
# imports

import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI
import json
import requests


In [17]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('GOOGLE_API_KEY')
RAPIDAPI_KEY = os.getenv('RAPIDAPI_KEY')
RAPIDAPI_HOST = os.getenv('RAPIDAPI_HOST')
# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [5]:
message = "Hello Gemini, how are you? My name is Sapphire smith. How rare is my name? "
messages = {"role":"user", "content":message}

print(messages)

{'role': 'user', 'content': 'Hello Gemini, how are you? My name is Sapphire smith. How rare is my name? '}


In [6]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini = OpenAI(base_url=GEMINI_BASE_URL,api_key=api_key)
response = gemini.chat.completions.create(model="gemini-2.5-flash-lite",messages = messages)
print(response.choices[0].message.content)


Hello Sapphire Smith! It's nice to meet you! As for your name, Sapphire is a **very rare and unique given name**.

Here's why:

*   **Not a Traditional Name:** "Sapphire" is primarily known as a precious gemstone. While it's beautiful and has a lovely meaning (associated with wisdom, royalty, and good fortune), it's not a name with a long history of common usage like Elizabeth or Mary.
*   **Gemstone Names are Uncommon:** While some parents are inspired by gemstones, they are still a relatively niche category of names.
*   **"Smith" is Very Common:** On the other hand, "Smith" is one of the most common surnames in English-speaking countries. This creates an interesting contrast with your unique first name.

**In summary, your first name, Sapphire, is quite rare, while your last name, Smith, is very common.** This combination makes you stand out!

It's a beautiful and distinctive name, Sapphire! Do you like how rare it is?


In [7]:
resume_text = """
I am a MERN Stack Developer with experience in React.js, Node.js, and MongoDB.
I have built full-stack applications using REST APIs.
I am familiar with JavaScript, HTML, CSS, and basic TypeScript.
"""

keyword_prompt = f"""
You are an AI assistant.

From the resume text below, extract the most important job-related keywords.
Focus on:
- Skills
- Technologies
- Job roles

Return the result as VALID JSON only.
Do not use markdown.
Do not include ``` or language names.

The response must be a JSON array of strings.

Resume:
{resume_text}
"""

In [8]:
messages = [
    {"role": "user", "content": keyword_prompt}
]

In [13]:
response = gemini.chat.completions.create(
    model="gemini-2.5-flash-lite",
    messages=messages
)
raw_keywords = response.choices[0].message.content
print(raw_keywords)

[
"MERN Stack Developer",
"React.js",
"Node.js",
"MongoDB",
"Full-stack applications",
"REST APIs",
"JavaScript",
"HTML",
"CSS",
"TypeScript"
]


In [14]:
# Parse the keywords from Gemini response
try:
    keywords_list = json.loads(raw_keywords)
    print("✅ Extracted Keywords:")
    for i, keyword in enumerate(keywords_list, 1):
        print(f"   {i}. {keyword}")
except json.JSONDecodeError as e:
    print(f"❌ Error parsing keywords: {e}")
    keywords_list = []

✅ Extracted Keywords:
   1. MERN Stack Developer
   2. React.js
   3. Node.js
   4. MongoDB
   5. Full-stack applications
   6. REST APIs
   7. JavaScript
   8. HTML
   9. CSS
   10. TypeScript


In [15]:
# Build the job title filter from keywords
job_titles = [kw for kw in keywords_list if any(role in kw.lower() for role in ['developer', 'engineer', 'stack', 'full'])]

# If no job titles found, use the first 3 technical skills
if not job_titles:
    job_titles = keywords_list[:3]

# Create the title filter with OR logic
title_filter = ' OR '.join([f'"{title}"' for title in job_titles])

# Indian tech hub locations
location_filter = '"Bangalore" OR "Mumbai" OR "Hyderabad" OR "Pune" OR "Chennai" OR "India"'

print(f"📋 Title Filter: {title_filter}")
print(f"📍 Location Filter: {location_filter}")

📋 Title Filter: "MERN Stack Developer" OR "Full-stack applications"
📍 Location Filter: "Bangalore" OR "Mumbai" OR "Hyderabad" OR "Pune" OR "Chennai" OR "India"


In [18]:
# RapidAPI LinkedIn Job Search API
url = "https://linkedin-job-search-api.p.rapidapi.com/active-jb-24h"

querystring = {
    "limit": "10",
    "offset": "0",
    "title_filter": title_filter,
    "location_filter": location_filter,
    "description_type": "text"
}

headers = {
    "x-rapidapi-key": f"{RAPIDAPI_KEY}",
    "x-rapidapi-host": RAPIDAPI_HOST
}

print("🔄 Fetching jobs from LinkedIn Job Search API...")

try:
    response = requests.get(url, headers=headers, params=querystring)
    response.raise_for_status()
    jobs_data = response.json()
    print(f"\n✅ API Response received successfully!")
except requests.exceptions.RequestException as e:
    print(f"❌ Error fetching jobs: {e}")
    jobs_data = []

🔄 Fetching jobs from LinkedIn Job Search API...

✅ API Response received successfully!


In [19]:
# Display the job results in a nice format
if isinstance(jobs_data, list) and len(jobs_data) > 0:
    print(f"\n🎯 Found {len(jobs_data)} job(s) matching your profile!\n")
    print("=" * 80)
    
    for i, job in enumerate(jobs_data, 1):
        title = job.get('title', 'N/A')
        company = job.get('company_name', job.get('company', 'N/A'))
        location = job.get('location', 'N/A')
        posted_date = job.get('posted_date', job.get('date', 'N/A'))
        job_url = job.get('linkedin_job_url', job.get('url', job.get('link', 'N/A')))
        description = job.get('description', '')[:300] + '...' if len(job.get('description', '')) > 300 else job.get('description', 'No description available')
        
        print(f"\n📌 Job #{i}")
        print(f"   🏷️  Title: {title}")
        print(f"   🏢 Company: {company}")
        print(f"   📍 Location: {location}")
        print(f"   📅 Posted: {posted_date}")
        print(f"   🔗 Link: {job_url}")
        print(f"   📝 Description: {description}")
        print("-" * 80)
else:
    print("\n⚠️ No jobs found matching your criteria.")
    print(f"\n📊 Raw API Response: {jobs_data}")


🎯 Found 7 job(s) matching your profile!


📌 Job #1
   🏷️  Title: MERN Stack Developer
   🏢 Company: N/A
   📍 Location: N/A
   📅 Posted: N/A
   🔗 Link: https://in.linkedin.com/jobs/view/mern-stack-developer-at-elitez-india-4362017353
   📝 Description: No description available
--------------------------------------------------------------------------------

📌 Job #2
   🏷️  Title: MERN Stack Developer
   🏢 Company: N/A
   📍 Location: N/A
   📅 Posted: N/A
   🔗 Link: https://in.linkedin.com/jobs/view/mern-stack-developer-at-lytegen-4354107150
   📝 Description: No description available
--------------------------------------------------------------------------------

📌 Job #3
   🏷️  Title: MERN Stack Developer Internship in Hyderabad
   🏢 Company: N/A
   📍 Location: N/A
   📅 Posted: N/A
   🔗 Link: https://in.linkedin.com/jobs/view/mern-stack-developer-internship-in-hyderabad-at-ziegler-aerospace-4364962957
   📝 Description: No description available
-------------------------------------------

In [20]:
# Summary Statistics
if isinstance(jobs_data, list) and len(jobs_data) > 0:
    companies = [job.get('company_name', job.get('company', 'Unknown')) for job in jobs_data]
    locations = [job.get('location', 'Unknown') for job in jobs_data]
    
    print("\n📊 Job Search Summary")
    print("=" * 40)
    print(f"   🔢 Total Jobs Found: {len(jobs_data)}")
    print(f"   🏢 Companies: {', '.join(set(companies))}")
    print(f"   📍 Locations: {', '.join(set(locations))}")
    print(f"   🔑 Keywords Used: {', '.join(keywords_list[:5])}...")
    print("\n✨ Good luck with your job search!")


📊 Job Search Summary
   🔢 Total Jobs Found: 7
   🏢 Companies: Unknown
   📍 Locations: Unknown
   🔑 Keywords Used: MERN Stack Developer, React.js, Node.js, MongoDB, Full-stack applications...

✨ Good luck with your job search!
